# MPS Tutorial
This sample will show you how to use the Aria MPS data via the MPS apis.
Please refer to the MPS wiki for more information about data formats and schemas

### Notebook stuck?
Note that because of Jupyter and Plotly issues, sometimes the code may stuck at visualization. We recommend **restart the kernels** and try again to see if the issue is resolved.


## Download the MPS sample dataset locally
> The sample dataset will get downloaded to a **tmp** folder by default. Please modify the path if necessary

In [ ]:
import subprocess, os

google_colab_env = 'google.colab' in str(get_ipython())
if google_colab_env:
    print("Running from Google Colab, installing projectaria_tools and getting sample data")
    mps_sample_path = "./mps_sample_data/"
else:
    mps_sample_path = "/tmp/mps_sample_data/"

base_url = "https://www.projectaria.com/async/sample/download/?bucket=mps&filename="
options = "-C - -O -L"

command_list = [
    f"mkdir -p {mps_sample_path}",
    # Getting sample data
    f'curl -o {mps_sample_path}/sample.vrs {options} "{base_url}sample.vrs"',
    f'curl -o {mps_sample_path}/trajectory.zip {options} "{base_url}trajectory.zip"',
    f'curl -o {mps_sample_path}/eye_gaze.zip {options} "{base_url}eye_gaze.zip"',
    # Unzip amd move the MPS output
    f"unzip -o {mps_sample_path}/eye_gaze.zip  -d {mps_sample_path}",
    f"unzip -o {mps_sample_path}/trajectory.zip -d {mps_sample_path}",
]

if google_colab_env:
  !pip install projectaria-tools
  for command in command_list:
    !$command
else:
  for command in command_list:
    subprocess.run(command, shell=True, check=True)

# Temporarily rename files to new schema, until we update the dataset on cloud
old_to_new = {
    f"trajectory/global_points.csv.gz": f"trajectory/semidense_points.csv.gz",
    f"eye_gaze/generalized_eye_gaze.csv": f"eye_gaze/general_eye_gaze.csv",
    f"eye_gaze/calibrated_eye_gaze.csv": f"eye_gaze/personalized_eye_gaze.csv",
}
for old, new in old_to_new.items():
  if os.path.isfile(os.path.join(mps_sample_path, old)):
    os.rename(os.path.join(mps_sample_path, old), os.path.join(mps_sample_path, new))

## Load the trajectory, point cloud and eye gaze using the MPS apis

In [ ]:
from projectaria_tools.core import data_provider, mps
from projectaria_tools.core.stream_id import StreamId
import numpy as np

# If necessary, set path to your own dataset and vrs file here
# mps_sample_path="/content/drive/MyDrive/MPS/sample_data"
# Load the VRS file
vrsfile = os.path.join(mps_sample_path, "sample.vrs")

# Trajectory and global points
closed_loop_trajectory = os.path.join(mps_sample_path, "trajectory","closed_loop_trajectory.csv")
semidense_points = os.path.join(mps_sample_path, "trajectory", "semidense_points.csv.gz")

# Eye Gaze
general_eye_gaze_path = os.path.join(mps_sample_path, "eye_gaze", "general_eye_gaze.csv")
personalized_eye_gaze_path = os.path.join(mps_sample_path, "eye_gaze", "personalized_eye_gaze.csv")
personalize_gaze_available = os.path.isfile(personalized_eye_gaze_path)

# Create data provider and get T_device_rgb
provider = data_provider.create_vrs_data_provider(vrsfile)
# Since we want to display the position of the RGB camera, we are querying its relative location
# from the device and will apply it to the device trajectory.
T_device_RGB = provider.get_device_calibration().get_transform_device_sensor("camera-rgb")

## Load trajectory and global points
mps_trajectory = mps.read_closed_loop_trajectory(closed_loop_trajectory)
points = mps.read_global_point_cloud(semidense_points, mps.StreamCompressionMode.GZIP)

## Load eyegaze
general_eye_gazes = mps.read_eyegaze(general_eye_gaze_path)
if personalize_gaze_available:
    personalized_eye_gazes = mps.read_eyegaze(personalized_eye_gaze_path)

## Helper functions

In [ ]:
import plotly.graph_objs as go
from matplotlib import pyplot as plt

# Helper function to build the frustum
def build_cam_frustum(transform_world_device):
    points = np.array([[0, 0, 0], [0.5, 0.5, 1], [-0.5, 0.5, 1], [-0.5, -0.5, 1], [0.5, -0.5, 1]]) * 0.6
    transform_world_rgb = transform_world_device @ T_device_RGB
    points_transformed = transform_world_rgb @ points.transpose()
    return go.Mesh3d(x=points_transformed[0,:], y=points_transformed[1, :], z=points_transformed[2, :],
        i=[0, 0, 0, 0, 1, 1], j=[1, 2, 3, 4, 2, 3], k=[2, 3, 4, 1, 3, 4], showscale=False, visible=False,
        colorscale="jet", intensity=points[:, 2], opacity=1.0, hoverinfo='none')

# Helper function to get nearest eye gaze for a timestamp
def get_nearest_eye_gaze(eye_gazes, query_timestamp_ns):
    return eye_gazes[min(range(len(eye_gazes)), key = lambda i: abs(eye_gazes[i].tracking_timestamp.total_seconds()*1e9 - query_timestamp_ns))]

# Helper function to project a gaze output onto the RGB image, assuming a fixed depth of 1m
def get_gaze_center_in_pixels(eye_gaze):
    depth_m = 1.0 # Select a fixed depth of 1m
    gaze_center_in_cpf = mps.get_eyegaze_point_at_depth(eye_gaze.yaw, eye_gaze.pitch, depth_m)
    transform_cpf_sensor = provider.get_device_calibration().get_transform_cpf_sensor(provider.get_label_from_stream_id(rgb_stream_id))
    gaze_center_in_camera = transform_cpf_sensor.inverse() @ gaze_center_in_cpf
    gaze_center_in_camera = gaze_center_in_camera / gaze_center_in_camera[2]
    gaze_center_in_pixels = cam_calibration.project(gaze_center_in_camera)
    return gaze_center_in_pixels

## Visualize the trajectory and point cloud in a 3D interactive plot
* Load trajectory
* Load global point cloud
* Render dense trajectory (1Khz) as points.
* Render subsampled 6DOF poses via camera frustum. Use calibration to transform RGB camera pose to world frame
* Render subsampled point cloud

_Please wait a minute for all the data to load. Zoom in to the point cloud and adjust your view. Then use the time slider to move the camera_

In [ ]:
# Load all world positions from the trajectory
traj = np.empty([len(mps_trajectory), 3])
for i in range(len(mps_trajectory)):
    traj[i, :] = mps_trajectory[i].transform_world_device.translation()

# Subsample trajectory for quick display
skip = 1000
mps_trajectory_subset = mps_trajectory[::skip]
steps = [None]*len(mps_trajectory_subset)

# Load each pose as a camera frustum trace
cam_frustums = [None]*len(mps_trajectory_subset)

for i in range(len(mps_trajectory_subset)):
    pose = mps_trajectory_subset[i]
    cam_frustums[i] = build_cam_frustum(pose.transform_world_device)
    timestamp = pose.tracking_timestamp.total_seconds()
    step = dict(method="update", args=[{"visible": [False] * len(cam_frustums) + [True] * 2}, {"title": "Trajectory and Point Cloud"},], label=timestamp,)
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps[i] = step
cam_frustums[0].visible = True

# Filter the point cloud by inv depth and depth and load
threshold_invdep = 0.001
threshold_dep = 0.15
skip = 1
point_cloud = np.empty([len(points), 3])
j = 0
for i in range(0, len(points), skip):
    if (points[i].inverse_distance_std < threshold_invdep and points[i].distance_std < threshold_dep):
        point_cloud[j,:] = points[i].position_world
        j=j+1
point_cloud = point_cloud[:j,:]

# Create slider to allow scrubbing and set the layout
sliders = [dict(currentvalue={"suffix": " s", "prefix": "Time :"}, pad={"t": 5}, steps=steps,)]
layout = go.Layout(sliders=sliders, scene=dict(bgcolor='lightgray', dragmode='orbit', aspectmode='data', xaxis_visible=False, yaxis_visible=False,zaxis_visible=False))

# Plot trajectory and point cloud
# We color the points by their z coordinate
trajectory = go.Scatter3d(x=traj[:, 0], y=traj[:, 1], z=traj[:, 2], mode="markers", marker={"size": 2, "opacity": 0.8, "color": "red"}, name="Trajectory", hoverinfo='none')
semidense_points = go.Scatter3d(x=point_cloud[:, 0], y=point_cloud[:, 1], z=point_cloud[:, 2], mode="markers",
    marker={"size" : 1.5, "color": point_cloud[:, 2], "cmin": -1.5, "cmax": 2, "colorscale": "viridis",},
    name="Global Points", hoverinfo='none')

# draw
plot_figure = go.Figure(data=cam_frustums + [trajectory, semidense_points], layout=layout)
plot_figure.show()

## Visualize general and personalized eye gaze projection on an rgb image.
* Load Eyegaze MPS output
* Select a random RGB frame
* Find the closest eye gaze data for the RGB frame
* Project the eye gaze for the RGB frame by **using a fixed depth of 1m**.
* Show the gaze cross on the RGB image

In [ ]:
rgb_stream_id = StreamId("214-1")
num_rgb_frames = provider.get_num_data(rgb_stream_id)
rgb_frame = provider.get_image_data_by_index(rgb_stream_id, (int)(num_rgb_frames-5))
assert rgb_frame[0] is not None, "no rgb frame"

image = rgb_frame[0].to_numpy_array()
capture_timestamp_ns = rgb_frame[1].capture_timestamp_ns
general_eye_gaze = get_nearest_eye_gaze(general_eye_gazes, capture_timestamp_ns)
if personalize_gaze_available:
    personalized_eye_gaze = get_nearest_eye_gaze(personalized_eye_gazes, capture_timestamp_ns)
# get projection function
device_calibration = provider.get_device_calibration()
cam_calibration = device_calibration.get_camera_calib(provider.get_label_from_stream_id(rgb_stream_id))
assert cam_calibration is not None, "no camera calibration"

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 10))

# Draw a cross at the projected gaze center location on the RGB image at depth 1m
general_gaze_center_in_pixels = get_gaze_center_in_pixels(general_eye_gaze)
if general_gaze_center_in_pixels is not None:
    ax1.imshow(image)
    ax1.plot(general_gaze_center_in_pixels[0], general_gaze_center_in_pixels[1], '+', c="red", mew=1, ms=20)
    ax1.set_title("Generalized Eye Gaze")
else:
    print(f"Eye gaze center projected to {general_gaze_center_in_pixels}, which is out of camera sensor plane.")

if personalize_gaze_available:
    personalized_gaze_center_in_pixels = get_gaze_center_in_pixels(personalized_eye_gaze)
    if personalized_gaze_center_in_pixels is not None:
        ax2.imshow(image)
        ax2.plot(personalized_gaze_center_in_pixels[0], personalized_gaze_center_in_pixels[1], '+', c="red", mew=1, ms=20)
        ax2.set_title("Personalized Eye Gaze")
    else:
        print(f"Eye gaze center projected to {personalized_gaze_center_in_pixels}, which is out of camera sensor plane.")
else:
    print("Personalized gaze not available for this dataset\n")
plt.show()
